## Initialize Fiddler Client


In [2]:
import fiddler as fdl

client = fdl.FiddlerApi()

## Load dataset


In [3]:
import pandas as pd
df = pd.read_csv('/app/fiddler_samples/samples/datasets/winequality/train.csv')
df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)


In [3]:
df_schema

DatasetInfo:
  display_name: 
  files: []
  columns:
                      column    dtype count(possible_values) is_nullable  \
    0                 row_id  INTEGER                              False   
    1          fixed acidity    FLOAT                              False   
    2       volatile acidity    FLOAT                              False   
    3            citric acid    FLOAT                              False   
    4         residual sugar    FLOAT                              False   
    5              chlorides    FLOAT                              False   
    6    free sulfur dioxide    FLOAT                              False   
    7   total sulfur dioxide    FLOAT                              False   
    8                density    FLOAT                              False   
    9                     pH    FLOAT                              False   
    10             sulphates    FLOAT                              False   
    11               alcohol    FLO

## Upload dataset

In [4]:
if 'wine_quality' not in client.list_datasets():
    upload_result = client.upload_dataset(
        dataset={'train': df}, 
        dataset_id='wine_quality')



## Create model schema

In [5]:
target = 'quality'
train_input = df.drop(columns=['row_id', 'quality'])
train_target = df[target]

feature_columns = list(train_input.columns)

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=client.get_dataset_info('wine_quality'),
    target=target, 
    features=feature_columns,
    display_name='sklearn model',
    description='this is a sklearn model from tutorial'
)

## Train model

In [6]:
import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing


regressor = sklearn.linear_model.LinearRegression()

full_model = sklearn.pipeline.Pipeline(steps=[
        ('standard_scaling', sklearn.preprocessing.StandardScaler()),
        ('model_name', regressor),
    ])

full_model.fit(train_input, train_target)
full_model.predict(train_input)


array([6.50506782, 6.65427237, 5.35208865, ..., 5.27678525, 6.36560789,
       4.44488096])

## Save model and schema

In [7]:
import pathlib
import shutil
import pickle
import yaml

project_id = 'tutorial'
model_id = 'wine_quality_model'

model_dir = pathlib.Path(model_id)
shutil.rmtree(model_dir, ignore_errors=True)
model_dir.mkdir()

with open(model_dir / 'model.pkl', 'wb') as pkl_file:
    pickle.dump(full_model, pkl_file)

with open(model_dir / 'model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)


## Write package.py wrapper

In [4]:
%%writefile wine_quality_model/package.py

import pickle
from pathlib import Path
import pandas as pd

PACKAGE_PATH = Path(__file__).parent

class SklearnModelPackage:
    is_classifier = False
    output_columns = ['predicted_quality']

    def __init__(self):
        with open(PACKAGE_PATH / 'model.pkl', 'rb') as infile:
            self.model = pickle.load(infile)

    def predict(self, input_df):
        f = self.model.predict if not self.is_classifier else self.model.predict_proba
        return pd.DataFrame(f(input_df), columns=self.output_columns)
    
def get_model():
    return SklearnModelPackage()



Overwriting wine_quality_model/package.py


In [5]:
!ls -l wine_quality_model

total 12
-rw-r--r-- 1 jovyan users    0 Sep 30 20:39 __init__.py
-rw-r--r-- 1 jovyan users 1408 Sep 30 20:21 model.pkl
-rw-r--r-- 1 jovyan users 1778 Sep 30 20:22 model.yaml
-rw-r--r-- 1 jovyan users  575 Sep 30 20:42 package.py


In [9]:
def my_import(name):
    components = name.split('.')
    mod = __import__(components[0])
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod

from importlib import reload

model_class = __import__('wine_quality_model.package')
reload(model_class)

model_class.get_model().predict(train_input[0:2])

AttributeError: module 'wine_quality_model' has no attribute 'get_model'

In [12]:
client.delete_model(project_id, model_id)
client.upload_model_package(model_dir, project_id, model_id)